* No EDA as there are so many other nb detailing it 
* No hyperparam optimization on the shallow models ... again there are some amazing nb explaining it. I checked a couple of shallow models for the baseline
* Just wanted to play with a NN on this issue
;-)


* RF = 0.7 ... NN = 0.717

In [ ]:

import numpy as np
import pandas as pd
from pathlib import Path
from operator import itemgetter    

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
import catboost as cb

import tensorflow as tf

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.utils import to_categorical

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')

# Data prep

In [ ]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')
print(test.shape)
test.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
# Any nulls ?

train.isnull().sum()

In [ ]:
# Separate target from train data

target = train.pop('target')
print(target.shape)
target.head()

In [ ]:
train.head()

### Normalize train and test separately so there will be no data leakage

In [ ]:
trainNorm = train.copy()
for feature_name in train.columns:
    mean_value = train[feature_name].mean()
    std_value = train[feature_name].std()
    trainNorm[feature_name] = (train[feature_name] - mean_value) / std_value
    
trainNorm.head()

In [ ]:
testNorm = test.copy()
for feature_name in test.columns:
    mean_value = test[feature_name].mean()
    std_value = test[feature_name].std()
    testNorm[feature_name] = (test[feature_name] - mean_value) / std_value
    
testNorm.head()

### Split train into train and validation

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(trainNorm, target, test_size=0.2, random_state=7)
print ('X_train: ', X_train.shape)
print ('X_test: ', X_test.shape)
print ('y_train: ', y_train.shape)
print ('y_test: ', y_test.shape)

# Models

In [ ]:
# CROSS VALIDATION

def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [ ]:
# Test Models and evaluation metric
# Lin reg ALL models HYPERPARAMS NOT optimized

models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(), SGDRegressor(), BayesianRidge(),
          cb.CatBoostRegressor(), RandomForestRegressor(), ]
names = ["LR", "Ridge", "Lasso", "ElasticNet", "SGD","BayesianRidge", "catboost","RandomForestRegressor"]

In [ ]:
%%time

# Run the models and compare

ModScores = {}

for name, model in zip(names, models):
    score = rmse_cv(model, X_train, y_train)
    ModScores[name] = score.mean()
    print("{}: {:.2f}".format(name,score.mean()))

print("_"*100)
for key, value in sorted(ModScores.items(), key = itemgetter(1), reverse = False):
    print(key, round(value,3))

### Final predict on test

In [ ]:
model = cb.CatBoostRegressor()

model.fit(X_train, y_train)

final_predictions = model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 
print("RMSE on X_test ", round(final_rmse, 4))

In [ ]:
# Model evaluation on test

final_predictions = model.predict(testNorm)
final_predictions.shape

subm = pd.read_csv(input_path / 'sample_submission.csv')
print(subm.shape)
subm.head()

id_col=subm.id
subm=id_col.to_frame()
subm['target'] = final_predictions
subm.set_index('id',inplace=True)
subm.head()

subm.to_csv('CBSubmission.csv')

LoadSub = pd.read_csv('CBSubmission.csv')
LoadSub.set_index('id',inplace=True)

LoadSub.head()

# NN

In [ ]:
# NN MODEL

from keras import models
model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],))) 
model.add(layers.Dense(256, activation='relu')) 
model.add(layers.Dropout(0.2)) 
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=optimizers.Adam(), loss='mse', metrics=['mae'])
print("model compiled")

In [ ]:
# Train NN

history = model.fit(X_train, y_train,
                    validation_split=0.2, 
                    verbose=1,
                   epochs=10)

In [ ]:
# Plot learning curves

acc = history.history['mae']
val_acc = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training error')
plt.plot(epochs, val_acc, 'r', label='Validation error')
plt.title('Training and validation ERROR')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation LOSS')
plt.legend()
plt.show()

In [ ]:
# Model evaluation on X_test

final_predictions = model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 
print("RMSE on X_test ", round(final_rmse, 4))

* Simple NN, 10 epochs RMSE = 0.7175

model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(1))

# Submission

In [ ]:
# Model evaluation on test

final_predictions = model.predict(testNorm)
final_predictions.shape

In [ ]:
subm = pd.read_csv(input_path / 'sample_submission.csv')
print(subm.shape)
subm.head()

In [ ]:
id_col=subm.id
subm=id_col.to_frame()
subm['target'] = final_predictions
subm.set_index('id',inplace=True)
subm.head()

In [ ]:
subm.to_csv('NNSubmission.csv')

In [ ]:
LoadSub = pd.read_csv('NNSubmission.csv')
LoadSub.set_index('id',inplace=True)

LoadSub.head()